# E-commerce Sales Forecasting & Customer Analytics Project

# Content
"This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers."

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('C:/Users/sawan/e-commerce_Project/data/raw/EcommerceData.csv' , encoding='latin1')

In [3]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [4]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [6]:
# Cleans and formats column names. for easy access and manipulation.
df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]  

# Data quality checks

In [7]:
# len(df['InvoiceNo'].unique())
df['invoiceno'].nunique()

25900

In [8]:
df.shape

(541909, 8)

In [9]:
df['stockcode'].nunique()

4070

# Check for missing values


In [10]:
df.isnull().sum()

invoiceno           0
stockcode           0
description      1454
quantity            0
invoicedate         0
unitprice           0
customerid     135080
country             0
dtype: int64

In [11]:
#check for duplicates
df.duplicated().sum()

np.int64(5268)

# Handling missing values

In [12]:
# Dropping rows with missing values in 'customerid' column and description column
df.dropna(subset=['customerid', 'description'], inplace=True)

In [13]:
df.isna().sum()

invoiceno      0
stockcode      0
description    0
quantity       0
invoicedate    0
unitprice      0
customerid     0
country        0
dtype: int64

# Feature Engineering


In [14]:

# Create a new column for total price
df['totalprice'] = df['quantity'] * df['unitprice'] 

In [15]:
# Change invoicedate to datetime format 
df['invoicedate']=pd.to_datetime(df['invoicedate'])

In [16]:
# Fixing duplicate entries
print(f"The dataset contains {df.duplicated().sum()} duplicate entries which needs to be removed.")
df.drop_duplicates(inplace=True)

The dataset contains 5225 duplicate entries which needs to be removed.


In [17]:
# Finding the number of unique stock codes
unique_stock_codes = df['stockcode'].nunique()

# Printing the number of unique stock codes
print(f"The number of unique stock codes in the dataset is: {unique_stock_codes}")

The number of unique stock codes in the dataset is: 3684


In [18]:
# Finding the number of numeric characters in each unique stock code
unique_stock_codes = df['stockcode'].unique()
unique_stock_codes

array(['85123A', '71053', '84406B', ..., '90214Z', '90089', '23843'],
      shape=(3684,), dtype=object)

In [19]:
numeric_char_counts_in_unique_codes = pd.Series(unique_stock_codes).apply(lambda x: sum(c.isdigit() for c in str(x))).value_counts()

# Printing the value counts for unique stock codes
print("Value counts of numeric character frequencies in unique stock codes:")
print("-"*70)
print(numeric_char_counts_in_unique_codes)

Value counts of numeric character frequencies in unique stock codes:
----------------------------------------------------------------------
5    3676
0       7
1       1
Name: count, dtype: int64


In [20]:
# We can see that most of the stock code have 5 digits which seems to be the standard format for stockcode but 8 of them are 0 or 1 

# finding and printing stock codes with 0 and 1 numerics
anomalous_stock_codes=[ code for code in unique_stock_codes if sum(c.isdigit() for c in str(code) )in (0,1)]

print("Anomalous stock codes :")
print('-'*22)
for code in anomalous_stock_codes:
    print(code)

Anomalous stock codes :
----------------------
POST
D
C2
M
BANK CHARGES
PADS
DOT
CRUK


In [21]:
# Based on the analysis the anamalous stock codes are only a small or negligible fraction in our dataset i.e. 8 out of 3684 stock codes
# So to clean our data and remove noise we must remove all these anomalous stock codes in our dataset

df= df[~df['stockcode'].isin(anomalous_stock_codes)]

In [22]:
df.shape

(399689, 9)

In [23]:
# Extrcting date features
df['date']= df['invoicedate'].dt.date
df['dayofweek']= df['invoicedate'].dt.dayofweek
# df['month']= df['invoicedate'].dt.month

In [24]:
df.head() 

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country,totalprice,date,dayofweek
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,2010-12-01,2
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12-01,2
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,2010-12-01,2
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12-01,2
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12-01,2


In [25]:
df.shape

(399689, 11)

In [26]:
# Treating zero unit prices
df.unitprice.describe()

count    399689.000000
mean          2.907457
std           4.451881
min           0.000000
25%           1.250000
50%           1.950000
75%           3.750000
max         649.500000
Name: unitprice, dtype: float64

In [27]:
len(df[df['unitprice']==0])

33

In [28]:
# This means that there are 33 records where unit price is 0 which means that ther are probably free items or possible data entry errors

# so to get better data without much noise we need to remove these entries
df=df[df['unitprice']>0]

In [29]:
df.head()

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country,totalprice,date,dayofweek
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,2010-12-01,2
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12-01,2
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,2010-12-01,2
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12-01,2
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12-01,2


# Using RFM features
RFM stands for:

Recency – How recently a customer made a purchase

Frequency – How often they purchase

Monetary – How much money they spend

It is a customer segmentation technique used in marketing and business analytics, especially in eCommerce, retail, and CRM.

# Recency

In [30]:
# Find the most recent purchase date for each customer
customer_data = df.groupby('customerid')['date'].max().reset_index()

# Find the most recent date in the entire dataset
most_recent_date = df['date'].max()

# Convert Date to datetime type before subtraction
customer_data['date'] = pd.to_datetime(customer_data['date'])
most_recent_date = pd.to_datetime(most_recent_date)

# Calculate the number of days since the last purchase for each customer
customer_data['Days_Since_Last_Purchase'] = (most_recent_date - customer_data['date']).dt.days

In [31]:
customer_data.head()

,customerid,date,Days_Since_Last_Purchase
0,12346.0,2011-01-18,325
1,12347.0,2011-12-07,2
2,12348.0,2011-09-25,75
3,12349.0,2011-11-21,18
4,12350.0,2011-02-02,310


In [32]:
# Remove the date column
# customer_data.drop(columns=['date'],inplace=True)+

# Frequency

In [33]:
# calculate the total number of purchases for each customer
purchases_per_customer = df.groupby('customerid')['totalprice'].nunique().reset_index()
purchases_per_customer.rename(columns={'invoiceno': 'total_purchases'}, inplace=True)
purchases_per_customer.head()

,customerid,totalprice
0,12346.0,2
1,12347.0,56
2,12348.0,14
3,12349.0,41
4,12350.0,8


# Monetary 

In [34]:
# calculate the total amount spent by each customer
totalspent= df.groupby('customerid')['totalprice'].sum().reset_index()
# totalspent.head()

#  Calculate the average transaction value for each customer
avg_transaction_value = df.groupby('customerid')['totalprice'].mean().reset_index()
avg_transaction_value.rename(columns={'totalprice': 'Average_Transaction_Value'}, inplace=True)
avg_transaction_value = pd.merge(avg_transaction_value ,totalspent, on='customerid')
avg_transaction_value = pd.merge(avg_transaction_value ,purchases_per_customer, on='customerid')

# Merge the new features into the customer_data dataframe
customer_data = pd.merge(customer_data, totalspent, on='customerid')
customer_data = pd.merge(customer_data, avg_transaction_value[['customerid', 'Average_Transaction_Value']], on='customerid')

In [35]:
customer_data

,customerid,date,Days_Since_Last_Purchase,totalprice,Average_Transaction_Value
0,12346.0,2011-01-18,325,0.00,0.000000
1,12347.0,2011-12-07,2,4310.00,23.681319
2,12348.0,2011-09-25,75,1437.24,53.231111
3,12349.0,2011-11-21,18,1457.55,20.243750
4,12350.0,2011-02-02,310,294.40,18.400000
...,...,...,...,...,...
4357,18280.0,2011-03-07,277,180.60,18.060000
4358,18281.0,2011-06-12,180,80.82,11.545714
4359,18282.0,2011-12-02,7,176.60,13.584615
4360,18283.0,2011-12-06,3,2039.58,2.836690


In [36]:
customer_data.to_csv('C:/Users/sawan/e-commerce_Project/data/processed/customer_data.csv', index=False)

# Save the processed data to a new CSV file

In [37]:
df.to_csv('../data/processed/ecommerce_data_clean.csv', index=False)

In [38]:
customer_data.head()

,customerid,date,Days_Since_Last_Purchase,totalprice,Average_Transaction_Value
0,12346.0,2011-01-18,325,0.00,0.000000
1,12347.0,2011-12-07,2,4310.00,23.681319
2,12348.0,2011-09-25,75,1437.24,53.231111
3,12349.0,2011-11-21,18,1457.55,20.243750
4,12350.0,2011-02-02,310,294.40,18.400000
